# Extração das regiões em tempo real

### Imports e outras inicializações

In [1]:
from io import BytesIO
from IPython.display import clear_output, Image, display
from PIL import Image as Img
import cv2
import numpy as np
import dlib
from scipy.spatial import distance as dist

### Extração em tempo real

In [2]:
classificador_dlib_68_pontos = './classificadores/shape_predictor_68_face_landmarks.dat'
classificador_dlib = dlib.shape_predictor(classificador_dlib_68_pontos)
detector_face = dlib.get_frontal_face_detector()

In [3]:
FACE = list(range(17, 68))
FACE_COMPLETA = list(range(0, 68))
LABIO = list(range(48, 61))
SOMBRANCELHA_DIREITA = list(range(17, 22))
SOMBRANCELHA_ESQUERDA = list(range(22, 27))
OLHO_DIREITO = list(range(36, 42))
OLHO_ESQUERDO = list(range(42, 48))
NARIZ = list(range(27, 35))
MANDIBULA = list(range(0, 17))


def padronizar_imagem(imagem):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    imagem = cv2.resize(imagem, (800, 600))
    return imagem


def exibir_video(frame):
    img = Img.fromarray(frame, 'RGB')
    buffer = BytesIO()
    img.save(buffer, format='JPEG')
    display(Image(data=buffer.getvalue()))
    clear_output(wait=True)


def pontos_marcos_faciais(imagem):
    retangulos = detector_face(imagem, 1)

    if len(retangulos) == 0:
        return None

    marcos = []
    for retangulo in retangulos:
        marcos.append(np.matrix([[p.x, p.y] for p in classificador_dlib(imagem, retangulo).parts()]))
    return marcos


def anotar_marcos_faciais(imagem, marcos_faciais):
    if imagem is None:
        return None

    for marco in marcos_faciais:
        for idx, ponto in enumerate(marco):
            centro = (ponto[0, 0], ponto[0, 1])
            cv2.circle(imagem, centro, 3, color=(255, 255, 0), thickness=-1)
    return imagem


def aspecto_razao_boca(pontos_boca):
    a = dist.euclidean(pontos_boca[3], pontos_boca[9])
    b = dist.euclidean(pontos_boca[2], pontos_boca[10])
    c = dist.euclidean(pontos_boca[4], pontos_boca[8])
    d = dist.euclidean(pontos_boca[0], pontos_boca[6])
    aspecto_razao = (a + b + c)/(3.0 * d)
    return aspecto_razao

In [4]:
captura_video = cv2.VideoCapture(0)

try:
    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            pontos = pontos_marcos_faciais(frame)
            if pontos is not None:
                frame = anotar_marcos_faciais(frame, pontos)
                pontos = pontos[0]
                ar = aspecto_razao_boca(pontos[LABIO])
                ar = round(ar, 3)

                if ar > 0.5:
                    (x, y, w, h) = cv2.boundingRect(pontos[LABIO])
                    frame = cv2.rectangle(
                        frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

                coord = tuple(pontos[LABIO][0].A1.reshape(1, -1)[0])
                coord = (coord[0], coord[1] + 20)
                cv2.putText(frame, 'Aspecto Razao ' + str(ar), coord,
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

            exibir_video(frame)

except KeyboardInterrupt:
    captura_video.release()
    print('Interrompido')

Interrompido
